## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
import numpy as np

A = np.random.randint(0, 1000, size=(1_000_000, 1))

def f1(A):
    acc, cnt = 0, 0
    for  ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc / cnt

%timeit f1(A)

1.68 s ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def f2(A):
    acc = 0
    for  ai in A:
        acc += ai + 100
    return acc / len(A)

%timeit f2(A)

1.55 s ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def f3(A):
    return sum(A) / len(A) + 100 

%timeit f3(A)

377 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%lprun -f f1 f1(A)

UsageError: Line magic function `%lprun` not found.


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [6]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

%timeit g(df)

536 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def g1(df):
    return df[df["key"].isin(('a', 'b', 'c', 'd', 'e'))]

%timeit g1(df)

93.1 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

In [8]:
!pip install line_profiler

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [9]:
import pandas as pd

In [10]:
recipes = pd.read_csv("recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')

reviews = pd.read_csv("reviews_sample.csv", sep=",", parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

In [11]:
def t_a():
    counter, values = 0, 0
    for index, row in reviews.iterrows():
        if row["date"].year == 2010:
            values += row["rating"]
            counter += 1

    return values/counter

example = t_a()

print(f"[Task 01 (A)] Answer: {example}")

[Task 01 (A)] Answer: 4.4544402182900615


In [12]:
%timeit t_a()

7.49 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
def t_b():
    counter, values = 0, 0
    selected_year_df = reviews[reviews['date'].dt.year == 2010]
    for index, row in selected_year_df.iterrows():
        values += row["rating"]
        counter += 1

    return values/counter
    
example2 = t_b()

print(f"[Task 01 (B)] Answer: {example2}")

[Task 01 (B)] Answer: 4.4544402182900615


In [14]:
%timeit t_b()

726 ms ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
def t_c():
    selected_year_df = reviews['date'].dt.year == 2010
    return reviews.loc[selected_year_df, 'rating'].mean()

example3 = t_c()
print(f"[Task 01 (C)] Answer: {example3}")

[Task 01 (C)] Answer: 4.4544402182900615


In [16]:
%timeit t_c()

11.3 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [17]:
%lprun -f t_a t_a()

UsageError: Line magic function `%lprun` not found.


In [18]:
%lprun -f t_b t_b()

UsageError: Line magic function `%lprun` not found.


In [19]:
%lprun -f t_c t_c()

UsageError: Line magic function `%lprun` not found.


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [20]:
import pandas as pd 

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count



In [21]:
get_word_reviews_count(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [22]:
%timeit get_word_reviews_count(reviews)

20.5 s ± 726 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%load_ext line_profiler
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 65.0989 s
File: C:\Users\user\AppData\Local\Temp/ipykernel_7808/1728842931.py
Function: get_word_reviews_count at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def get_word_reviews_count(df):
     4         1         12.0     12.0      0.0      word_reviews = {}
     5    126680  204186083.0   1611.8     31.4      for _, row in df.dropna(subset=['review']).iterrows():
     6    126679   28863605.0    227.8      4.4          recipe_id, review = row['recipe_id'], row['review']
     7    126679    5718348.0     45.1      0.9          words = review.split(' ')
     8   6918689   21365313.0      3.1      3.3          for word in words:
     9   6792010   31865834.0      4.7      4.9              if word not in word_reviews:
    10    174944     953485.0      5.5      0.1                  word_reviews[word] = []
    11   6792010   34660135.0      5.1      5.3              word_reviews[w

In [24]:
def get_word_reviews_count_optimized(df):
    
    word_reviews = {}
    
    #Удаляем нулевые и проходим идерацией
    for _, row in df.dropna(subset=['review']).iterrows():

        #По каждому слову начинаем цикл
        for word in row['review'].split(' '):
            #Если слово не в словаре, то заносим его
            if word not in word_reviews:
                word_reviews[word] = 0
            #Добавляем рецепт по этому слову
            word_reviews[word] += 1
    
    return word_reviews

In [25]:
get_word_reviews_count_optimized(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [26]:
%timeit get_word_reviews_count_optimized(reviews)

9.98 s ± 306 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%lprun -f get_word_reviews_count_optimized get_word_reviews_count_optimized(reviews)

Timer unit: 1e-07 s

Total time: 29.7843 s
File: C:\Users\user\AppData\Local\Temp/ipykernel_7808/2522783595.py
Function: get_word_reviews_count_optimized at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count_optimized(df):
     2                                               
     3         1         14.0     14.0      0.0      word_reviews = {}
     4                                               
     5                                               #Удаляем нулевые и проходим идерацией
     6    126680  196360789.0   1550.1     65.9      for _, row in df.dropna(subset=['review']).iterrows():
     7                                           
     8                                                   #По каждому слову начинаем цикл
     9   6918689   41497263.0      6.0     13.9          for word in row['review'].split(' '):
    10                                                       #Если сл

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [28]:
buffer = reviews[['recipe_id', 'rating']].dropna()
mask = buffer['rating'] != 0
buffer = buffer[mask].groupby(buffer['recipe_id'])['rating']

In [ ]:
#Удаление из выборки отзывов с нулевым рейтингом

In [29]:
def executor(series_data, function):
    """Метод для вызова функции c аргументами"""
    return function(series_data.to_numpy(), series_data.mean())

In [30]:
def MAPE_FIRST(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

#Конвертация dataframe в series
result1 = buffer.agg(executor, MAPE_FIRST)

In [31]:
%timeit buffer.agg(executor, MAPE_FIRST)

1.99 s ± 98.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
@numba.jit(nopython=True)
def MAPE_SECOND(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

result2 = buffer.agg(executor, MAPE_SECOND)

NameError: name 'numba' is not defined

In [ ]:
%timeit buffer.agg(executor, MAPE_SECOND)